# Affichage et exploration des données de tennis de table

Ce notebook est dédié à l'exploration visuelle des données issues des matchs.  
Il permet notamment de visualiser les vidéos des points isolés, d'afficher des figures statiques ou interactives, et d'explorer les clusters de rebonds sur la table.

---

## 1. Import des modules et connexion à la base de données

On commence par importer les modules nécessaires et se connecter à la base de données.

In [1]:
import sqlite3
import sys
import os

chemin_dossier_racine = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(chemin_dossier_racine)

from modules.data_processing import *
from modules.data_interpretation import *
from modules.figure_maker import *
from modules.figure_module import *

# Connexion à la base de données SQLite
con = sqlite3.connect("BDD_avec_cluster.db")

---

## 2. Affichage de figure statique

La méthode `Figure.show()` permet l'affichage statique de données sur une table de tennis de table.
Ici, on récupère les positions de service d'un joueur sur deux matchs, puis on les affiche.

In [2]:
# Récupération des données de service pour un joueur sur deux matchs
requete_services = (
    "SELECT * FROM Liste_des_coups "
    "WHERE num_coup = 1 "
    "AND (Gamename='FAN-ZHENDONG_vs_ALEXIS-LEBRUN' OR Gamename='ALEXIS-LEBRUN_vs_FAN-ZHENDONG') "
    "AND joueur_frappe='ALEXIS-LEBRUN'"
)

df_service = pd.read_sql_query(requete_services, con)

# Extraction des coordonnées des services (on filtre les valeurs NaN)
positions_services = [
    (df_service["coor_balle_x"][i], df_service["coor_balle_y"][i])
    for i in range(len(df_service["coor_balle_x"]))
    if not math.isnan(df_service["coor_balle_x"][i])
]

# Placement des rebonds du bon côté de la table
_, positions_services, _, _ = place_on_side([], positions_services, [], [])

In [3]:
# Affichage des positions de service sur une figure statique
figure = Figure()
figure.add_positions_rebonds([positions_services], [0])
figure.show()

---

## 3. Affichage interactif des données pour tous les joueurs

La méthode `Figure.show_global_test_with_video()` permet d'afficher une figure interactive associée à un menu déroulant pour choisir le joueur, l'effet, et le numéro de coup.  
On y retrouve également la table de contingence et la p-value du test du $\chi^2$.

In [4]:
# Récupération des listes pour le menu déroulant interactif
joueurs = figure.get_list_joueurs()  # Liste des joueurs
effets = figure.get_list_effets()  # Liste des effets
num_coups = figure.get_list_num_coups()  # Liste des numéros de coups
joueur_initial = joueurs[0]  # Joueur par défaut

# Affichage interactif avec Dash
figure.show_global_test_with_video(joueurs, joueur_initial, effets, num_coups)

L'application est disponible à l'adresse suivante : http://127.0.0.1:59078


---

## 4. Affichage interactif des vidéos associées aux points

La méthode `Figure.show_data_with_video()` permet d'afficher la figure et d'accéder aux vidéos des points associés.

In [5]:
figure.show_data_with_video()

L'application est disponible à l'adresse suivante : http://127.0.0.1:59081
